In [1]:
from pynq import Overlay
from pynq import Xlnk
from pynq.lib import AxiGPIO
import numpy as np
# import pynq.lib.dma
import time
# import math
import matplotlib.pyplot as plt

mask = 0xffffffff
xlnk = Xlnk()

In [2]:
ol = Overlay('./project_dma_mux_cnt.bit')

In [3]:
# DMA
dma = ol.axi_dma_0

# AXI GPIO
gpio_resetn_ip = ol.ip_dict['axi_gpio_resetn']
rx_rstn = AxiGPIO(gpio_resetn_ip).channel1

gpio_mux_ip = ol.ip_dict['axi_gpio_mux']
mux_sel = AxiGPIO(gpio_mux_ip).channel1

gpio_capture_ip = ol.ip_dict['DataPath/R_GPIO_group/axi_gpio_capture']
channel_sw = AxiGPIO(gpio_capture_ip).channel1
last_cnt = AxiGPIO(gpio_capture_ip).channel2

In [4]:
print('Reset state:', rx_rstn.read())
print('Mux state', mux_sel.read())
print('Channel off state:', channel_sw.read())
print('Capture size default:', last_cnt.read())

Reset state: 0
Mux state 0
Channel off state: 0
Capture size default: 4096


In [ ]:
def Cpature_size(Number: int, last_cnt):
    last_cnt.write(Number, mask)
    print("Capture size:", last_cnt.read())

In [ ]:
dma.recvchannel._mmio.write(0x30, 0x0004) #soft reset
dma.recvchannel.start()
dma.sendchannel.start()

# FROM DMA

In [5]:
Data = ' National Central University\n Department of Communication Engineering\n '
chars = [ord(c) for c in Data]  # ord() => return ascii code
buf_size = len(chars)
print(buf_size)

71


In [6]:
channel_sw.write(0x0, mask)
rx_rstn.write(0x0, mask)
rx_rstn.write(0x1, mask)
mux_sel.write(0x0, mask)

CaptureNumber = buf_size
last_cnt.write(CaptureNumber, mask)
print('Capture size:', last_cnt.read())

output_buffer = xlnk.cma_array(shape=(CaptureNumber,), dtype=np.uint32)
dma.recvchannel.transfer(output_buffer)

channel_sw.write(0x1, mask)  #channel on
input_buffer = xlnk.cma_array(shape=(buf_size,), dtype=np.uint32)
input_buffer[:] = chars
print(input_buffer[0:50])
dma.sendchannel.transfer(input_buffer)
dma.sendchannel.wait()


dma.recvchannel.wait()
channel_sw.write(0x0, mask)  #channel off

Capture size: 71
[ 32  78  97 116 105 111 110  97 108  32  67 101 110 116 114  97 108  32
  85 110 105 118 101 114 115 105 116 121  10  32  68 101 112  97 114 116
 109 101 110 116  32 111 102  32  67 111 109 109 117 110]


In [7]:
print(type(output_buffer))
print(output_buffer)

<class 'pynq.xlnk.ContiguousArray'>
[ 32  78  97 116 105 111 110  97 108  32  67 101 110 116 114  97 108  32
  85 110 105 118 101 114 115 105 116 121  10  32  68 101 112  97 114 116
 109 101 110 116  32 111 102  32  67 111 109 109 117 110 105  99  97 116
 105 111 110  32  69 110 103 105 110 101 101 114 105 110 103  10  32]


In [8]:
back_data = [chr(s) for s in output_buffer]
back_data_str = "".join(back_data)
print(back_data_str)
print(type(back_data_str))

 National Central University
 Department of Communication Engineering
 
<class 'str'>


# Input: Counter (hardware)

In [9]:
channel_sw.write(0x0, mask)
rx_rstn.write(0x0, mask)
rx_rstn.write(0x1, mask)
mux_sel.write(0x1, mask)

CaptureNumber = 2**12
last_cnt.write(CaptureNumber, mask)
print('Capture size:', last_cnt.read())

output_buffer = xlnk.cma_array(shape=(CaptureNumber,), dtype=np.uint32)
dma.recvchannel.transfer(output_buffer)

channel_sw.write(0x1, mask)  #channel on


dma.recvchannel.wait()
channel_sw.write(0x0, mask)  #channel off

Capture size: 4096


In [10]:
print(output_buffer[1:50])

[2648895281 2648895282 2648895283 2648895284 2648895285 2648895286
 2648895287 2648895288 2648895289 2648895290 2648895291 2648895292
 2648895293 2648895294 2648895295 2648895296 2648895297 2648895298
 2648895299 2648895300 2648895301 2648895302 2648895303 2648895304
 2648895305 2648895306 2648895307 2648895308 2648895309 2648895310
 2648895311 2648895312 2648895313 2648895314 2648895315 2648895316
 2648895317 2648895318 2648895319 2648895320 2648895321 2648895322
 2648895323 2648895324 2648895325 2648895326 2648895327 2648895328
 2648895329]
